# Create Elastic Search Pipeline

Create and wire containers:
* elasticsearch
* kibana
* filebeats
* logstash

## Create Elasticsearch Container

This section replicates the function of this docker cli command
```bash
docker run -d -p 9200:9200 -p 9300:9300 \
--network=elastic \
--name=elasticsearch \
-v /Users/lou/Documents/elasticsearch/esdata1:/usr/share/elasticsearch/data \
-e "node.name=alacrity.local" \
--restart=unless-stopped docker.elastic.co/elasticsearch/elasticsearch:6.3.2
```

In [1]:
import docker
import socket
import uuid
client = docker.from_env()

In [2]:
# replace these with the appropriate values for your setup
# all components in pipeline should match versions
es_version = '6.3.2'
# configurations and data for docker containers are rooted here
docker_root = '/Users/lou/Documents/docker-python-api-elk-provisioning'

In [3]:
# we need to create a random elastic search container name since we
# can't currently directly create an alias. See https://github.com/docker/docker-py/issues/1571
es_container_name="elasticsearch-{}".format(str(uuid.uuid4()))
es_container_name

'elasticsearch-1b832754-c4a5-43d9-995e-4fe9324271b3'

In [4]:
es_image = "docker.elastic.co/elasticsearch/elasticsearch:{}".format(es_version)
es_container = client.containers.run(
    es_image,
    name=es_container_name,
    detach=True,
    network="elastic",
    ports={
        '9200/tcp': '9200',
        '9300/tcp': '9300'
    },
    volumes={
        "{}/elasticsearch/esdata1".format(docker_root): {
            'bind': '/usr/share/elasticsearch/data',
            'mode': 'rw'
        }
    },
    environment={
        'node.name': socket.gethostname()
    },
    restart_policy={
        'Name': 'unless-stopped'
    }
)


## Create Kibana Container
This section replicates the function of this docker cli command
```bash
docker run -d -p 5601:5601 \
--network=elastic \
-e "ELASTICSEARCH_URL=http://elasticsearch:9200" \
-v /Users/lou/Documents/kibana/kibana.yml:/usr/share/kibana/config/kibana.yml \
--restart=unless-stopped docker.elastic.co/kibana/kibana:6.3.2
```

In [5]:
kb_image = "docker.elastic.co/kibana/kibana:{}".format(es_version)
kb_container = client.containers.run(
    kb_image,
    detach=True,
    network="elastic",
    environment={
        'ELASTICSEARCH_URL': 'http://{}:9200'.format(es_container_name)
    },
    restart_policy={
        'Name': 'unless-stopped'
    },
    ports={
        '5601/tcp': '5601'
    },
    volumes={
        "{}/kibana/kibana.yml".format(docker_root): {
            'bind': '/usr/share/kibana/config/kibana.yml',
            'mode': 'ro'
        }
    }
)

## Create Logstash Container

In [40]:
ls_container_name="logstash-{}".format(str(uuid.uuid4()))
ls_image = "docker.elastic.co/logstash/logstash:{}".format(es_version)
ls_container = client.containers.run(
    ls_image,
    name=ls_container_name,
    detach=True,
    network='elastic',
    restart_policy={
        'Name': 'unless-stopped'
    },
    environment={
        'ELASTICSEARCH_HOST': '{}:9200'.format(es_container_name)
    },
    volumes={
        "{}/logstash/pipeline/".format(docker_root): {
            'bind': '/usr/share/logstash/pipeline/',
            'mode': 'rw'
        },
        "{}/logstash/config/".format(docker_root): {
            'bind': '/usr/share/logstash/config/',
            'mode': 'ro'
        },
        "{}/logstash/data".format(docker_root): {
            'bind': '/usr/share/logstash/data',
            'mode': 'rw'
        }
    } 
)

## Create Filebeat Container

The filebeat.yml and the volume specification should be modified to match where filebeat should
pick up log files to process. 

In [41]:
fb_image = "docker.elastic.co/beats/filebeat:{}".format(es_version)
fb_container = client.containers.run(
    fb_image,
    detach=True,
    network='elastic',
    restart_policy={
        'Name': 'unless-stopped'
    },
    environment={
        'LOGSTASH_HOST': '{}:5044'.format(ls_container_name)
    },
    volumes={
        "{}/filebeat/filebeat.yml".format(docker_root): {
            'bind': '/usr/share/filebeat/filebeat.yml',
            'mode': 'ro'
        },
        "{}/filebeat/inputlogs".format(docker_root):{
            'bind': '/usr/share/filebeat/inputlogs',
            'mode': 'ro'
        },
        "{}/filebeat/data".format(docker_root):{
            'bind': '/usr/share/filebeat/data',
            'mode': 'rw'
        }
    }    
    
)

### TODO
* root access for filebeat.yml
* environment variable for LOGSTASH_HOST
* clean data registry to re-run logs/files into filebeat /data/registry

## Shutdown and Clean Up

In [42]:
ls_container.stop()
ls_container.remove()
fb_container.stop()
fb_container.remove()
kb_container.stop()
kb_container.remove()
es_container.stop()
es_container.remove()



In [43]:
!docker ps

CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS                    NAMES
ced9158e6147        influxdb            "/entrypoint.sh -con…"   4 weeks ago         Up 26 hours         0.0.0.0:8086->8086/tcp   snow-influxdb
854c7db780df        mysql:latest        "docker-entrypoint.s…"   5 weeks ago         Up 2 days           0.0.0.0:3306->3306/tcp   snow-mysql
